In [12]:
import pandas as pd

In [2]:
#read CSV file and put it into a DataFrame called covidDataDF
covidDataDF = pd.read_csv("WHO-COVID-19-global-daily-data.csv")

#filters covidDataDF to only include rows which include 'United States of America' and stores it in a new DataFrame called USCovidData
USCovidData = covidDataDF[covidDataDF['Country'] == 'United States of America'].copy()

#uses pandas functiton of 'to_datetime' to convert the date reported into a datetime object so that information like date, month, and year can be easily pulled from it 
USCovidData['Date_reported'] = pd.to_datetime(USCovidData['Date_reported'])

#creates a new column called 'Month_number' in the DataFrame which then returns the month number of the month that is being stores in 'Date_reported'
USCovidData['Month_number'] = USCovidData['Date_reported'].dt.month
#creates a new column called 'Month' in the DataFrame which then returns the full name of the month that is being stored in 'Date_reported'
USCovidData['Month'] = USCovidData['Date_reported'].dt.month_name()
#creates a new column called 'Year' in the DataFrame which then returns the year that is being stored in 'Date_reported' as a string
USCovidData['Year'] = USCovidData['Date_reported'].dt.year.astype(str)

#orders the columns of the DataFrame so that the new 'Year' and 'Month' columns are between 'Date_reported' and 'Country_code'
USCovidData = USCovidData[['Date_reported', 'Year', 'Month_number', 'Month', 'Country_code', 'Country', 'WHO_region', 'New_cases', 'Cumulative_cases', 'New_deaths', 'Cumulative_deaths']]

In [3]:
#this groups the US Covid data by year, month number, and month. Month number is ahead of month because it will be sorted in alphabetical order if Month is first, and we want chronological order
USDataByMonth = USCovidData.groupby(['Year', 'Month_number', 'Month'])

#.agg() is a method within GroupBy and in this code it is telling Pandas that for each group (from above), to combine the rows in different ways for each column
#since we need the sum for new cases and new deaths and the last data point for cumulative cases and deaths, we have to use this method 
#'sum' adds up all the new cases and new deaths within each group to get a monthly total
#'last' reports the last value in cumulative cases and cumulative deaths. we are able to use 'last' since the cumulative total will always be highest on the last item in the group
USMonthlyCovidData = USDataByMonth.agg({
    'New_cases': 'sum',
    'Cumulative_cases': 'last',
    'New_deaths': 'sum',
    'Cumulative_deaths': 'last'
})

#because we used the groupby function and changed how the table headers were set up, we reset the index to only include columns for the data we need and to order them in a specific way
USMonthlyCovidData = USMonthlyCovidData.reset_index().sort_values(['Year', 'Month_number'])

#convert Year, Month_number, New_cases, Cumulative_cases, New_deaths, and Cumulative_deaths so that all are ints
USMonthlyCovidData['Year'] = USMonthlyCovidData['Year'].astype(int)
USMonthlyCovidData['Month_number'] = USMonthlyCovidData['Month_number'].astype(int)
USMonthlyCovidData['New_cases'] = USMonthlyCovidData['New_cases'].astype(int)
USMonthlyCovidData['Cumulative_cases'] = USMonthlyCovidData['Cumulative_cases'].astype(int)
USMonthlyCovidData['New_deaths'] = USMonthlyCovidData['New_deaths'].astype(int)
USMonthlyCovidData['Cumulative_deaths'] = USMonthlyCovidData['Cumulative_deaths'].astype(int)

USMonthlyCovidData

,Year,Month_number,Month,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020,1,January,8,8,0,0
1,2020,2,February,61,69,1,1
2,2020,3,March,173074,173143,3326,3327
3,2020,4,April,857163,1030306,57134,60461
4,2020,5,May,755116,1785422,44864,105325
...,...,...,...,...,...,...,...
61,2025,2,February,0,103436829,3016,1219837
62,2025,3,March,0,103436829,2059,1221896
63,2025,4,April,0,103436829,1771,1223667
64,2025,5,May,0,103436829,823,1224490


In [13]:
#read CSV file and put it into a dataframe called USTravelDF
USTravelDF = pd.read_csv("USTravelData.csv")

#the column names are not consistent across the two data sets, so this renames them to match the syntax of the covid data
USTravelDF = USTravelDF.rename(columns={
    'Month Number': 'Month_number',
    'Foreign Originating': 'Foreign_originating',
    'Foreign Returning': 'Foreign_returning',
    'U.S. Citizen Originating': 'US_citizen_originating',
    'U.S. Citizen Returning': 'US_citizen_returning'
})

#the columns in each seperate csv store data as different types, so they all need to be changed to match the covid data
USTravelDF['Year'] = USTravelDF['Year'].astype(int)

#the travel data is in the wrong order, so this changes the order to be the same as the covid data
USTravelDF = USTravelDF.sort_values(['Year', 'Month_number', 'Month'])

#merge the two datasets into one
combinedCovidTravelData = pd.merge(USMonthlyCovidData, USTravelDF, on = ['Year', 'Month_number', 'Month'])

#created a new column called Timestamp which takes in the year and month and converts it into a year-month timestamp
combinedCovidTravelData['Timestamp'] = pd.to_datetime(
    combinedCovidTravelData['Year'].astype(str) + '-' + combinedCovidTravelData['Month_number'].astype(str)
).dt.to_period('M')

#saves the new DataFrame as a combined csv file
combinedCovidTravelData.to_csv("US_Covid_and_Travel_Data.csv", index = False)

combinedCovidTravelData

,Year,Month_number,Month,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,Foreign_originating,Foreign_returning,US_citizen_originating,US_citizen_returning,Timestamp
0,2020,1,January,8,8,0,0,5274405,5520357,4073096,5218812,2020-01
1,2020,2,February,61,69,1,1,4470729,4448720,3902937,4155775,2020-02
2,2020,3,March,173074,173143,3326,3327,1960896,2946059,1844726,3053096,2020-03
3,2020,4,April,857163,1030306,57134,60461,54031,176627,32144,110285,2020-04
4,2020,5,May,755116,1785422,44864,105325,78337,192614,54368,107630,2020-05
...,...,...,...,...,...,...,...,...,...,...,...,...
60,2025,1,January,0,103436829,3948,1216821,4848091,5236859,4970681,6434329,2025-01
61,2025,2,February,0,103436829,3016,1219837,4115637,4170631,4912048,5036802,2025-02
62,2025,3,March,0,103436829,2059,1221896,4540667,4708418,6557399,6739342,2025-03
63,2025,4,April,0,103436829,1771,1223667,5040051,4822189,5919390,6334486,2025-04
